In [1]:
import os

In [2]:
%pwd

'/Users/ntchindagiscard/Documents/end-end-mlflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/ntchindagiscard/Documents/end-end-mlflow'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataingestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unsizp_dir: Path


In [ ]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
